In [1]:
import cv2
import numpy as np 
from datetime import date
from deepface import DeepFace     
from mss import mss
import time


In [ ]:
# Load Haar cascade for face detection.
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Optionally, precompute your reference embeddings.
# For example, you might have a folder "reference_faces" with subfolders or images named with the label.
# You can run this once at startup to build your embeddings database.
# Alternatively, DeepFace.find() can search a folder directly.
db_path = r"C:\Users\User\Pictures\Face_recog_db"
def detect_and_recognize(screen_frame):
    # Convert frame to grayscale for face detection.
    gray_image = cv2.cvtColor(screen_frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))



    faces_info = DeepFace.extract_faces(
            img_path=screen_frame,
            detector_backend="mtcnn",
            enforce_detection=False,
            anti_spoofing=False
        )
    
    for face_info in faces_info:
        # Extract the face ROI and the facial area information.
        face_roi = face_info["face"] 
        facial_area = face_info["facial_area"]
        x = facial_area["x"]
        y = facial_area["y"]
        w = facial_area["w"]
        h = facial_area["h"]


        # Extract face region of interest (ROI)
        face_roi = screen_frame[y:y+h, x:x+w]
        # Convert ROI from BGR to RGB (DeepFace expects RGB images)
        face_roi_rgb = face_roi ##cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        
        try:
            # Use DeepFace.find() to compare the detected face against your reference database.
            # Setting enforce_detection=False to avoid errors if a face isn't perfectly aligned.
            
            
            results = DeepFace.find(img_path=face_roi_rgb, db_path=db_path, model_name= "Facenet", enforce_detection=False)

            ##result_ver = DeepFace.verify(face_roi_rgb, db_path, model_name="VGG-Face")

            ##Label = result_ver["verified"]


            
            
            if len(results) > 0 and len(results[0]) > 0:
                # Assuming results is a list of DataFrames (one per model)
                # Get the first match's identity, then extract the name from the file path.
                identity_path = results[0].iloc[0]["identity"]
                label = identity_path.split("/")[-1].split("\\")[-1]  # works for Linux/Mac and Windows paths
            else:
                label = "Unknown"
        except Exception as e:
            # If DeepFace fails to process, label as Unknown.
            label = "Unknown_from_exception"
            print("Recognition error:", e)
        
        # Draw rectangle and label on the frame.
        cv2.rectangle(screen_frame, (x, y), (x+w, y+h), (0, 255, 0), 4)
        cv2.putText(screen_frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return screen_frame

# Capture the screen feed from the second monitor.
with mss() as sct:
    monitor = sct.monitors[2]
    while True:
        # Grab the screen image.
        img = np.array(sct.grab(monitor))
        # Convert image from BGRA to BGR.
        screen_frame = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        
        # Start the timer
        start_time = time.time()

        # Detect faces and perform recognition.
        processed_frame = detect_and_recognize(screen_frame)

        # End the timer
        end_time = time.time()
        processing_time = end_time - start_time
        print(f"Processing time: {processing_time:.4f} seconds")
        
        cv2.imshow("My Face Detection & Recognition Project", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()

25-03-26 20:30:17 - Searching [[[ 61  59  64]
  [ 58  54  59]
  [ 57  54  59]
  ...
  [ 42  34  37]
  [ 41  36  38]
  [ 41  38  40]]

 [[ 60  58  64]
  [ 59  55  61]
  [ 60  58  63]
  ...
  [ 46  39  42]
  [ 49  44  46]
  [ 47  43  45]]

 [[ 67  67  73]
  [ 69  67  74]
  [ 65  63  70]
  ...
  [ 49  42  47]
  [ 52  46  49]
  [ 49  45  47]]

 ...

 [[110 135 176]
  [110 136 172]
  [113 136 173]
  ...
  [222 224 225]
  [222 224 225]
  [222 224 225]]

 [[111 137 178]
  [113 137 172]
  [114 136 171]
  ...
  [224 226 227]
  [223 225 226]
  [223 225 226]]

 [[110 135 177]
  [114 135 172]
  [117 138 174]
  ...
  [224 226 227]
  [221 223 224]
  [222 224 225]]] in 5 length datastore
25-03-26 20:30:18 - find function duration 0.5264468193054199 seconds
25-03-26 20:30:18 - Searching [[[ 30  31  31]
  [ 29  29  29]
  [ 32  31  30]
  ...
  [ 16  16  15]
  [  7   4   4]
  [ 16  12  13]]

 [[ 27  29  29]
  [ 19  20  20]
  [ 36  35  34]
  ...
  [ 35  36  39]
  [ 22  20  22]
  [ 25  21  24]]

 [[ 80  83